## 기본설정 및 함수정의

#### - 목표 : 
#### 1. 분석대상지에서 분석한 탄소배출 및 흡수 요인을 바탕으로 적용대상지의 토지이용계획, 추정 교통량, 계획인구 등의 데이터를 활용해<br>용도별 탄소배출량 및 흡수량을 인공지능 기법을 활용해 추정
#### 2. 적용대상지의 기존 계획으로 추정된 탄소배출량 및 흡수량을 기준으로 미래 모빌리티 분담률 적용에 따른 탄소저감 효과를 분석하고,<br>이를 공원·녹지조성 효과(면적)로 산정
#### 3. 적용대상지인 광명시흥 공공주택지구의 미래모빌리티 도입에 따른 탄소저감 효과분석에 관한 내용을 반드시 분석 보고서에 포함
##### “DRT 운행 통계” 데이터, “광명시흥 교통수요 예측” 보고서를 활용하여 미래 모빌리티 분담률에 따른 탄소저감 효과를 자유롭게 분석하여 추정하되,<br>분석대상 모빌리티는 육상 수단만을 대상으로 함(UAM 제외)


In [29]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# tqdm의 pandas전용 메소드를 호출
tqdm.pandas()
# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)
 
# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.progress_apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

In [31]:
# 송파구 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_11240_2023_4Q/bnd_dong_11240_2023_4Q.shp"
songpa_gdf = gpd.read_file(shapefile_path1)
songpa_gdf = songpa_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 인천서구 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_23080_2023_4Q/bnd_dong_23080_2023_4Q.shp"
incheon_seo_gdf = gpd.read_file(shapefile_path1)
incheon_seo_gdf = incheon_seo_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 성남시 수정구 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_31021_2023_4Q/bnd_dong_31021_2023_4Q.shp"
sungnam_soojung_gdf = gpd.read_file(shapefile_path1)
sungnam_soojung_gdf = sungnam_soojung_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# # 성남시 중원구 행정동 데이터
# shapefile_path1 = "SBJ_2504_001/bnd_dong_31022_2023_4Q/bnd_dong_31022_2023_4Q.shp"
# sungnam_jungwon_gdf = gpd.read_file(shapefile_path1)
# sungnam_jungwon_gdf = sungnam_jungwon_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# # 성남시 분당구 행정동 데이터
# shapefile_path1 = "SBJ_2504_001/bnd_dong_31023_2023_4Q/bnd_dong_31023_2023_4Q.shp"
# sungnam_bundang_gdf = gpd.read_file(shapefile_path1)
# sungnam_bundang_gdf = sungnam_bundang_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#  하남시 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_31180_2023_4Q/bnd_dong_31180_2023_4Q.shp"
hanam_gdf = gpd.read_file(shapefile_path1)
hanam_gdf = hanam_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#  화성시 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_31240_2023_4Q/bnd_dong_31240_2023_4Q.shp"
hwasung_gdf = gpd.read_file(shapefile_path1)
hwasung_gdf = hwasung_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 광명시 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_31060_2023_4Q/bnd_dong_31060_2023_4Q.shp"
gwangmyung_gdf = gpd.read_file(shapefile_path1)
gwangmyung_gdf = gwangmyung_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 시흥시 행정동 데이터
shapefile_path1 = "SBJ_2504_001/bnd_dong_31150_2023_4Q/bnd_dong_31150_2023_4Q.shp"
siheung_gdf = gpd.read_file(shapefile_path1)
siheung_gdf = siheung_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 필요한 GeoDataFrame들을 리스트로 묶어서 병합
gdf_list = [
    songpa_gdf, incheon_seo_gdf, sungnam_soojung_gdf,
    hanam_gdf, hwasung_gdf, gwangmyung_gdf, siheung_gdf
]

# 하나의 GeoDataFrame으로 통합
merged_area_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs="EPSG:4326")

##### 적용대상지 격자

In [32]:
# GeoJSON 파일 불러오기
with open('SBJ_2504_001/10. 격자(적용대상지).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_apply = pd.json_normalize(geojson_data['features'])
grid_apply['geometry'] = grid_apply['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_apply.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_apply 데이터프레임을 GeoDataFrame으로 변환
grid_apply = gpd.GeoDataFrame(grid_apply, geometry='geometry')
grid_apply_dict = dict(zip(grid_apply['properties.gid'], grid_apply['geometry']))

# 제거할 열 리스트
drop_cols = ['type', 'geometry.type']
grid_apply = grid_apply.drop(columns=drop_cols)

# 'properties.' 접두사 제거
grid_apply.columns = [
    col.replace('properties.', '') if col.startswith('properties.') else col
    for col in grid_apply.columns
]

grid_apply = grid_apply.drop_duplicates(subset='gid', keep='first')

# 격자와 도시공간의 공간 교차
grid_apply = gpd.sjoin(grid_apply, merged_area_gdf, how='inner', predicate='intersects')

# 필요 없는 열 제거 (sjoin 결과에는 index_right 등 추가됨)
grid_apply = grid_apply.drop(columns=["index_right"])
grid_apply = grid_apply.drop_duplicates(['gid'], keep='last')
grid_apply = grid_apply[['gbn', 'gid', 'geometry']].reset_index(drop=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5612\1966219583.py:24: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326




In [ ]:
# 맵 객체 생성 및 데이터 로드
grid_map = KeplerGl(height=1000, width=1500)
grid_map.add_data(data=grid_apply, name="격자 데이터")
grid_map.add_data(data=gwangmyung_gdf, name="광명 데이터")
grid_map.add_data(data=siheung_gdf, name="시흥 데이터")

# 맵 출력 및 상세설정
grid_map

In [ ]:
# 맵 저장
# grid_map.save_to_html(file_name="visualization/화성시 현황/화성시 격자거주인구 map.html")

Map saved to visualization/화성시 현황/화성시 격자거주인구 map.html!


#### 토지이용계획도

In [ ]:
areaplan = gpd.read_file('SBJ_2504_001/11._토지이용계획도.geojson', encoding='euc-kr')
areaplan = gpd.GeoDataFrame(areaplan, geometry='geometry')
areaplan = areaplan[['zoneName', 'blockName', 'blockType', 'geometry']]

# 분석대상 & 적용대상 구분
areaplan_analysis = areaplan[areaplan['zoneName']!='광명시흥 공공주택지구']
areaplan_analysis.loc[areaplan_analysis['blockName'] == 'D42', ['blockName', 'blockType']] = ['도', '도로']
gwangmyeong_siheung = areaplan[areaplan['zoneName']=='광명시흥 공공주택지구']

# 엑셀 파일 불러오기
plan_info = pd.read_excel("SBJ_2504_001/26. (참고자료) 토지이용_구역별계획.xlsx")

groupings = {
    "1": ["D2-1", "D2-2"],  # 건설호수 96, 인구 239
    "2": ["D1-1", "D1-2", "D1-3", "D1-4", "D1-5", "D1-6", "D1-7", "D1-8", "D1-9", "D1-10", "D1-11", "D1-12", "D1-13", "D1-14",
          "D2-3", "D2-4", "D2-5", "D2-6", "D2-7", "D2-8", "D2-9", "D2-10", "D2-11"],  # 971, 2428
    "3": ["BD1-1", "BD1-2", "BD2-1", "BD2-2"],  # 건설호수 161, 인구 403
}
group_values = {
    "1": {"건설호수": 96, "인구": 239},
    "2": {"건설호수": 971, "인구": 2428},
    "3": {"건설호수": 161, "인구": 403},
}

# 엑셀 데이터에 건설호수/인구 배분
plan_info["건설호수_추정"] = None
plan_info["인구_추정"] = None

for group, zones in groupings.items():
    if group not in group_values:
        continue  # 값 없는 그룹은 생략

    subset = plan_info[plan_info["구역"].isin(zones)].copy()
    total_area = subset["면적"].sum()
    if total_area == 0:
        continue

    # 비율 계산
    area_ratio = subset["면적"] / total_area
    plan_info.loc[plan_info["구역"].isin(zones), "건설호수_추정"] = (area_ratio * group_values[group]["건설호수"]).round().astype("Int64")
    plan_info.loc[plan_info["구역"].isin(zones), "인구_추정"] = (area_ratio * group_values[group]["인구"]).round().astype("Int64")

# 기존 코드 이후에 이 블록만 추가
for group, zones in groupings.items():
    if group not in group_values:
        continue  # 값이 없는 그룹은 생략

    # 건설호수_추정 → 건설호수, 인구_추정 → 인구
    plan_info.loc[plan_info["구역"].isin(zones), "건설호수"] = plan_info.loc[plan_info["구역"].isin(zones), "건설호수_추정"]
    plan_info.loc[plan_info["구역"].isin(zones), "인구"] = plan_info.loc[plan_info["구역"].isin(zones), "인구_추정"]

# 필요한 열만 선택
columns_to_merge = ['구역', '면적', '건설호수', '인구', '주택유형']
plan_info = plan_info[columns_to_merge]

# 병합 수행 ('blockName' <-> '구역' 기준, left join으로 기존 areaplan_target에 추가 열 붙이기)
gwangmyeong_siheung = gwangmyeong_siheung.merge(plan_info, how='left', left_on='blockName', right_on='구역')
gwangmyeong_siheung['건설호수'] = gwangmyeong_siheung['건설호수'].astype('Int64')
gwangmyeong_siheung['인구'] = gwangmyeong_siheung['인구'].astype('Int64')

# 불필요한 '구역' 열 제거 (원하는 경우)
gwangmyeong_siheung.drop(columns='구역', inplace=True)

In [38]:
dongtan = areaplan_analysis[(areaplan_analysis['zoneName'] == '화성동탄지구 택지개발사업')|(areaplan_analysis['zoneName'] == '화성동탄2지구 택지개발예정지구')]
dongtan_polygon = unary_union(dongtan.geometry)

wirye = areaplan_analysis[areaplan_analysis['zoneName'] == '위례 택지개발사업 개발계획']
wirye_polygon = unary_union(wirye.geometry)

cheongna = areaplan_analysis[areaplan_analysis['zoneName'] == '인천경제자유구역 청라국제도시']
cheongna_polygon = unary_union(cheongna.geometry)

misa = areaplan_analysis[areaplan_analysis['zoneName'] == '하남미사 공공주택지구 조성사업']
misa_polygon = unary_union(misa.geometry)

gwangmyeong_siheung_polygon = unary_union(gwangmyeong_siheung.geometry)

# 필요한 GeoDataFrame들을 리스트로 묶어서 병합
gdf_list = [
    dongtan, wirye, cheongna,
    misa, gwangmyeong_siheung
]

# 하나의 GeoDataFrame으로 통합
merged_area_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs="EPSG:4326")
merged_area_gdf = merged_area_gdf[['zoneName','blockName','blockType','geometry']]

In [ ]:
# 맵 객체 생성 및 데이터 로드
areaplan_map = KeplerGl(height=1000, width=1500)
areaplan_map.add_data(data=gwangmyeong_siheung, name="토지이용계획 데이터")

# 맵 출력 및 상세설정
areaplan_map

#### 교차로 추정교통량

In [ ]:
e_crossroad_2032 = gpd.read_file('SBJ_2504_001/24. 교차로_추정교통량_2032.geojson', encoding='euc-kr')
e_crossroad_2032 = gpd.GeoDataFrame(e_crossroad_2032, geometry='geometry')

e_crossroad_2036 = gpd.read_file('SBJ_2504_001/25. 교차로_추정교통량_2036.geojson', encoding='euc-kr')
e_crossroad_2036 = gpd.GeoDataFrame(e_crossroad_2036, geometry='geometry')

In [ ]:
# 맵 객체 생성 및 데이터 로드
areaplan_map = KeplerGl(height=1000, width=1500)
areaplan_map.add_data(data=e_crossroad_2032, name="데이터")

# 맵 출력 및 상세설정
areaplan_map